# Board games
### Data Engineering Capstone Project

#### Project Summary
In this project we are going to receive details about board games from two sources. The first source contains meta information about almost 100.000 board games (number of players, genre, ..). The second source contains the ratings of the board games on a daily basis (+2 mio. rows). Our goal is to link these two sources and transform the data into an easier digestible data model to enable Data Analysts to see trends by genre or other game parameters.

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

##### Code to receive the two datasets. Change these cells to Code to run them. The files in this project were last updated on 25 05 2020.

##### Imports, global settings and initializing session

In [1]:
import configparser
import os
import pandas as pd
import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, explode, input_file_name, isnan, monotonically_increasing_id, split, substring_index, to_date, when
from pyspark.sql.types import BooleanType, StructType, IntegerType, DoubleType, StringType, StructField

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    
    return spark

In [3]:
pd.set_option('display.max_columns', None)
spark = create_spark_session()

### Step 1: Scope the Project and Gather Data

#### Scope 
We are transforming the two data sources about board games and their ratings into a data model with five tables. We gather the daily ratings of the board games, aswell as category, game family, game mechanics, number of players, minimum age and some more.
Since we talk about daily ratings, we can then use this data to find trending game types in the board gaming industry.

The amount of data is still easily managable on a local machine, so we decided to run this locally while (thinking about possible future scenarios) using mainly Spark for reading the data and transforming it. In the end we will persist the tables with parquet.
 
#### Describe and Gather Data 
First dataset (Daily rankings; given as git repository with csv files):
https://github.com/beefsack/bgg-ranking-historicals 

Second dataset (Meta Data; given as kaggle download as sqlite file):
https://www.kaggle.com/mshepherd/board-games?select=bga_GameItem.csv

In [4]:
ranking_data = "./bgg-ranking-historicals/*.csv"

df_staging_ranking_unfiltered = spark.read.option("header", "true") \
                                   .option("inferSchema", "true") \
                                   .csv(ranking_data) \
                                   .withColumn("filename", input_file_name())

In [5]:
# it seems like pyspark does not support reading from sqlite db file (easily). Since the dataset is quite small, we use pandas to load the data and convert to a spark dataframe afterwards
# in case the data in this file increases a lot, we will need to find a better way to digest the data
conn = sqlite3.connect('./bgg-meta/database.sqlite')

df_staging_meta_pd = pd.read_sql_query("SELECT * FROM BoardGames", conn)
df_staging_meta_unfiltered = spark.createDataFrame(df_staging_meta_pd)

In [6]:
df_staging_ranking_unfiltered.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Rank: integer (nullable = true)
 |-- Average: double (nullable = true)
 |-- Bayes average: double (nullable = true)
 |-- Users rated: integer (nullable = true)
 |-- URL: string (nullable = true)
 |-- Thumbnail: string (nullable = true)
 |-- filename: string (nullable = false)



In [7]:
df_staging_meta_unfiltered.printSchema()

root
 |-- row_names: string (nullable = true)
 |-- game.id: string (nullable = true)
 |-- game.type: string (nullable = true)
 |-- details.description: string (nullable = true)
 |-- details.image: string (nullable = true)
 |-- details.maxplayers: double (nullable = true)
 |-- details.maxplaytime: double (nullable = true)
 |-- details.minage: double (nullable = true)
 |-- details.minplayers: double (nullable = true)
 |-- details.minplaytime: double (nullable = true)
 |-- details.name: string (nullable = true)
 |-- details.playingtime: double (nullable = true)
 |-- details.thumbnail: string (nullable = true)
 |-- details.yearpublished: double (nullable = true)
 |-- attributes.boardgameartist: string (nullable = true)
 |-- attributes.boardgamecategory: string (nullable = true)
 |-- attributes.boardgamecompilation: string (nullable = true)
 |-- attributes.boardgamedesigner: string (nullable = true)
 |-- attributes.boardgameexpansion: string (nullable = true)
 |-- attributes.boardgamefamily

### Step 2: Explore and Assess the Data
#### Exploring the data and cleaning steps - Meta data

##### It is a bit tedious to use spark with dots in column names. So we replace dots with underscores in the first step

In [8]:
for col in df_staging_meta_unfiltered.columns:
    df_staging_meta_unfiltered = df_staging_meta_unfiltered.withColumnRenamed(col, col.replace('.', '_'))

In [9]:
df_staging_meta_unfiltered.limit(5).toPandas()

,row_names,game_id,game_type,details_description,details_image,details_maxplayers,details_maxplaytime,details_minage,details_minplayers,details_minplaytime,details_name,details_playingtime,details_thumbnail,details_yearpublished,attributes_boardgameartist,attributes_boardgamecategory,attributes_boardgamecompilation,attributes_boardgamedesigner,attributes_boardgameexpansion,attributes_boardgamefamily,attributes_boardgameimplementation,attributes_boardgameintegration,attributes_boardgamemechanic,attributes_boardgamepublisher,attributes_total,stats_average,stats_averageweight,stats_bayesaverage,stats_family_abstracts_bayesaverage,stats_family_abstracts_pos,stats_family_cgs_bayesaverage,stats_family_cgs_pos,stats_family_childrensgames_bayesaverage,stats_family_childrensgames_pos,stats_family_familygames_bayesaverage,stats_family_familygames_pos,stats_family_partygames_bayesaverage,stats_family_partygames_pos,stats_family_strategygames_bayesaverage,stats_family_strategygames_pos,stats_family_thematic_bayesaverage,stats_family_thematic_pos,stats_family_wargames_bayesaverage,stats_family_wargames_pos,stats_median,stats_numcomments,stats_numweights,stats_owned,stats_stddev,stats_subtype_boardgame_bayesaverage,stats_subtype_boardgame_pos,stats_trading,stats_usersrated,stats_wanting,stats_wishing,polls_language_dependence,polls_suggested_numplayers_1,polls_suggested_numplayers_10,polls_suggested_numplayers_2,polls_suggested_numplayers_3,polls_suggested_numplayers_4,polls_suggested_numplayers_5,polls_suggested_numplayers_6,polls_suggested_numplayers_7,polls_suggested_numplayers_8,polls_suggested_numplayers_9,polls_suggested_numplayers_Over,polls_suggested_playerage,attributes_t_links_concat_2____,stats_family_amiga_bayesaverage,stats_family_amiga_pos,stats_family_arcade_bayesaverage,stats_family_arcade_pos,stats_family_atarist_bayesaverage,stats_family_atarist_pos,stats_family_commodore64_bayesaverage,stats_family_commodore64_pos,stats_subtype_rpgitem_bayesaverage,stats_subtype_rpgitem_pos,stats_subtype_videogame_bayesaverage,stats_subtype_videogame_pos
0,1,1,boardgame,Die Macher is a game about seven sequential po...,//cf.geekdo-images.com/images/pic159509.jpg,5.0,240.0,14.0,3.0,240.0,Die Macher,240.0,//cf.geekdo-images.com/images/pic159509_t.jpg,1986.0,Marcus Gschwendtner,"Economic,Negotiation,Political",None,Karl-Heinz Schmiel,None,"Country: Germany,Valley Games Classic Line",None,None,"Area Control / Area Influence,Auction/Bidding,...","Hans im Glück Verlags-GmbH,Moskito Spiele,Vall...",6.0,7.66508,4.3477,7.29168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.39570,85.0,NaN,NaN,NaN,NaN,0.0,1763.0,719.0,5251.0,1.59321,7.29168,147.0,170.0,4498.0,505.0,1654.0,No,NotRecommended,None,NotRecommended,NotRecommended,Recommended,Best,None,None,None,None,NotRecommended,14,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,boardgame,Dragonmaster is a trick-taking card game based...,//cf.geekdo-images.com/images/pic184174.jpg,4.0,30.0,12.0,3.0,30.0,Dragonmaster,30.0,//cf.geekdo-images.com/images/pic184174_t.jpg,1981.0,Bob Pepper,"Card Game,Fantasy",None,"G. W. ""Jerry"" D'Arcey",None,Animals: Dragons,"Indulgence,Coup d'etat",None,Trick-taking,"E.S. Lowe,Milton Bradley",7.0,6.60815,1.9423,5.87150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.91318,1066.0,NaN,NaN,NaN,NaN,0.0,273.0,52.0,1053.0,1.46282,5.87150,2541.0,73.0,478.0,67.0,161.0,Some,NotRecommended,None,NotRecommended,Recommended,Best,None,None,None,None,None,NotRecommended,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,boardgame,"Part of the Knizia tile-laying trilogy, Samura...",//cf.geekdo-images.com/images/pic3211873.jpg,4.0,60.0,10.0,2.0,30.0,Samurai,60.0,//cf.geekdo-images.com/images/pic3211873_t.jpg,1998.0,Franz Vohwinkel,"Abstract Strategy,Medieval",None,Reiner Knizia,None,"Asian Theme,Country: Japan,Knizia tile-laying ...",None,None,"Area Control / Area Influence,Hand Management,...","999 Games,ABACUSSPIELE,Astrel Games,Ceilikan J...",6.0,7.44119,2.5085,7.28295,NaN

##### Looks fine on first glance. We will filter for columns that are not related to rankings / statistics (we will get the daily rankings from bgg-ranking-historicals). We are mainly interested in details about player number, time etc.

In [10]:
FILTER_COLS = ['game_id',                       \
               'details_name',                  \
               'game_type',                     \
               'details_maxplayers',            \
               'details_maxplaytime',           \
               'details_minage',                \
               'details_minplayers',            \
               'details_minplaytime',           \
               'details_playingtime',           \
               'details_yearpublished',         \
               'attributes_boardgameartist',    \
               'attributes_boardgamecategory',  \
               'attributes_boardgamedesigner',  \
               'attributes_boardgamefamily',    \
               'attributes_boardgamemechanic'   ]

df_staging_meta = df_staging_meta_unfiltered.select(FILTER_COLS)

In [11]:
df_staging_meta.limit(5).toPandas()

,game_id,details_name,game_type,details_maxplayers,details_maxplaytime,details_minage,details_minplayers,details_minplaytime,details_playingtime,details_yearpublished,attributes_boardgameartist,attributes_boardgamecategory,attributes_boardgamedesigner,attributes_boardgamefamily,attributes_boardgamemechanic
0,1,Die Macher,boardgame,5.0,240.0,14.0,3.0,240.0,240.0,1986.0,Marcus Gschwendtner,"Economic,Negotiation,Political",Karl-Heinz Schmiel,"Country: Germany,Valley Games Classic Line","Area Control / Area Influence,Auction/Bidding,..."
1,2,Dragonmaster,boardgame,4.0,30.0,12.0,3.0,30.0,30.0,1981.0,Bob Pepper,"Card Game,Fantasy","G. W. ""Jerry"" D'Arcey",Animals: Dragons,Trick-taking
2,3,Samurai,boardgame,4.0,60.0,10.0,2.0,30.0,60.0,1998.0,Franz Vohwinkel,"Abstract Strategy,Medieval",Reiner Knizia,"Asian Theme,Country: Japan,Knizia tile-laying ...","Area Control / Area Influence,Hand Management,..."
3,4,Tal der Könige,boardgame,4.0,60.0,12.0,2.0,60.0,60.0,1992.0,None,Ancient,Christian Beierer,"Country: Egypt,Promotional Board Games","Action Point Allowance System,Area Control / A..."
4,5,Acquire,boardgame,6.0,90.0,12.0,3.0,90.0,90.0,1964.0,"Scott Okumura,Peter Whitley",Economic,Sid Sackson,3M Bookshelf Series,"Hand Management,Stock Holding,Tile Placement"


##### Note that the columns details_maxplayers, details_maxplaytime, details_minage, details_minplayers, details_minplaytime, details_playingtime are given as double. We will cast to integer.

In [12]:
COL_CAST_TO_INT = ['details_maxplayers',   \
                   'details_maxplaytime',  \
                   'details_minage',       \
                   'details_minplayers',   \
                   'details_minplaytime',  \
                   'details_playingtime',  \
                   'details_yearpublished' ]

for col in COL_CAST_TO_INT:
    df_staging_meta = df_staging_meta.withColumn(col, df_staging_meta[col].cast("int"))

In [13]:
df_staging_meta.limit(5).toPandas()

,game_id,details_name,game_type,details_maxplayers,details_maxplaytime,details_minage,details_minplayers,details_minplaytime,details_playingtime,details_yearpublished,attributes_boardgameartist,attributes_boardgamecategory,attributes_boardgamedesigner,attributes_boardgamefamily,attributes_boardgamemechanic
0,1,Die Macher,boardgame,5,240,14,3,240,240,1986,Marcus Gschwendtner,"Economic,Negotiation,Political",Karl-Heinz Schmiel,"Country: Germany,Valley Games Classic Line","Area Control / Area Influence,Auction/Bidding,..."
1,2,Dragonmaster,boardgame,4,30,12,3,30,30,1981,Bob Pepper,"Card Game,Fantasy","G. W. ""Jerry"" D'Arcey",Animals: Dragons,Trick-taking
2,3,Samurai,boardgame,4,60,10,2,30,60,1998,Franz Vohwinkel,"Abstract Strategy,Medieval",Reiner Knizia,"Asian Theme,Country: Japan,Knizia tile-laying ...","Area Control / Area Influence,Hand Management,..."
3,4,Tal der Könige,boardgame,4,60,12,2,60,60,1992,None,Ancient,Christian Beierer,"Country: Egypt,Promotional Board Games","Action Point Allowance System,Area Control / A..."
4,5,Acquire,boardgame,6,90,12,3,90,90,1964,"Scott Okumura,Peter Whitley",Economic,Sid Sackson,3M Bookshelf Series,"Hand Management,Stock Holding,Tile Placement"


##### Count number of NaN values.

In [14]:
df_staging_meta.select([count(when(isnan(col), col)).alias(col) for col in df_staging_meta.columns]).toPandas()

,game_id,details_name,game_type,details_maxplayers,details_maxplaytime,details_minage,details_minplayers,details_minplaytime,details_playingtime,details_yearpublished,attributes_boardgameartist,attributes_boardgamecategory,attributes_boardgamedesigner,attributes_boardgamefamily,attributes_boardgamemechanic
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Looks fine. Let's also look at basic stats

In [15]:
meta_described = df_staging_meta.describe()

In [16]:
meta_described[meta_described.columns[:10]].show()

+-------+-----------------+------------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+
|summary|          game_id|            details_name|         game_type|details_maxplayers|details_maxplaytime|    details_minage|details_minplayers|details_minplaytime|details_playingtime|
+-------+-----------------+------------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+
|  count|            90400|                   90400|             90400|             90400|              90400|             90400|             90400|              90400|              90400|
|   mean| 88667.6744358407|                Infinity|              null|5.7318141592920355|  51.77201327433628| 7.120907079646018| 1.982466814159292|  46.10297566371681|  51.77201327433628|
| stddev|70143.68048686023|                     NaN|   

In [17]:
meta_described[meta_described.columns[10:]].show()

+---------------------+--------------------------+----------------------------+----------------------------+--------------------------+----------------------------+
|details_yearpublished|attributes_boardgameartist|attributes_boardgamecategory|attributes_boardgamedesigner|attributes_boardgamefamily|attributes_boardgamemechanic|
+---------------------+--------------------------+----------------------------+----------------------------+--------------------------+----------------------------+
|                90400|                     34665|                       88869|                       78517|                     50168|                       75163|
|   1813.9963827433628|                      null|                        null|                         7.7|                      null|                        null|
|    580.7408401570714|                      null|                        null|                         NaN|                      null|                        null|
|         

##### Looks fine (almost). Apparently there is at least one entry which does not carry a valid yearpublished.

In [18]:
df_staging_meta.filter("details_yearpublished < 0").count()

22

In [19]:
df_staging_meta.filter("details_yearpublished < 0").limit(5).toPandas()

,game_id,details_name,game_type,details_maxplayers,details_maxplaytime,details_minage,details_minplayers,details_minplaytime,details_playingtime,details_yearpublished,attributes_boardgameartist,attributes_boardgamecategory,attributes_boardgamedesigner,attributes_boardgamefamily,attributes_boardgamemechanic
0,188,Go,boardgame,2,180,8,2,30,180,-2200,"Barbro Hennius,Norman Sommer",Abstract Strategy,(Uncredited),"3M Bookshelf Series,Combinatorial,Ravensburger...",Area Enclosure
1,1602,The Royal Game of Ur,boardgame,2,30,10,2,30,30,-2500,None,"Abstract Strategy,Ancient,Dice,Racing",(Uncredited),"Country: Iraq,The Reliquary Collection","Dice Rolling,Roll / Spin and Move"
2,2397,Backgammon,boardgame,2,30,8,2,30,30,-3000,"(Uncredited),Willem Cornelisz Duyster","Abstract Strategy,Dice",(Uncredited),"3M Bookshelf Series,Characters: Austin Powers,...","Betting/Wagering,Roll / Spin and Move"
3,2399,Senet,boardgame,2,30,6,2,30,30,-3500,Johan Andreasson,"Abstract Strategy,Dice,Religious",(Uncredited),"Continent: Africa,Country: Egypt,Gammon","Dice Rolling,Roll / Spin and Move"
4,2912,Hounds and Jackals: Game of the Pharaohs,boardgame,2,20,6,2,20,20,-2000,None,"Ancient,Dice,Racing",(Uncredited),"Animals: Dogs,Country: Egypt",Roll / Spin and Move


##### Seemed wrong at first glance but looking at the entries, negative values make sense for really old games like Go.

#### Cleaning Steps - Ranking data

In [20]:
df_staging_ranking_unfiltered.select([count(when(isnan(col), col)).alias(col) for col in df_staging_ranking_unfiltered.columns]).show()

+---+----+----+----+-------+-------------+-----------+---+---------+--------+
| ID|Name|Year|Rank|Average|Bayes average|Users rated|URL|Thumbnail|filename|
+---+----+----+----+-------+-------------+-----------+---+---------+--------+
|  0|   0|   0|   0|      0|            0|          0|  0|        0|       0|
+---+----+----+----+-------+-------------+-----------+---+---------+--------+



##### No NaN values in df_ranking so no steps necessary.

In [21]:
ranking_described = df_staging_ranking_unfiltered.describe()

In [22]:
ranking_described.show()

+-------+----------------+------------------------+------------------+-----------------+------------------+-------------------+------------------+--------------------+--------------------+--------------------+
|summary|              ID|                    Name|              Year|             Rank|           Average|      Bayes average|       Users rated|                 URL|           Thumbnail|            filename|
+-------+----------------+------------------------+------------------+-----------------+------------------+-------------------+------------------+--------------------+--------------------+--------------------+
|  count|         2243307|                 2243307|           2243307|          2243307|           2243307|            2243307|           2243307|             2243307|             2240165|             2243307|
|   mean|92974.4586920114|                Infinity|1986.0322889377155|8976.399270362906| 6.373496824108263|  5.694996599663177| 787.2662783114394|              

In [23]:
df_staging_ranking_unfiltered.filter("Year > 2020").count()

687

In [24]:
df_staging_ranking_unfiltered.filter("Year > 2020").limit(5).toPandas()

,ID,Name,Year,Rank,Average,Bayes average,Users rated,URL,Thumbnail,filename
0,188,Go,2200,152,7.64,7.340,13715,/boardgame/188/go,https://cf.geekdo-images.com/micro/img/ncEmbZZ...,file:/home/workspace/bgg-ranking-historicals/2...
1,2397,Backgammon,3000,1217,6.53,6.358,10664,/boardgame/2397/backgammon,https://cf.geekdo-images.com/micro/img/bWZaJZy...,file:/home/workspace/bgg-ranking-historicals/2...
2,2399,Senet,3500,6751,5.90,5.598,590,/boardgame/2399/senet,https://cf.geekdo-images.com/micro/img/RMaALim...,file:/home/workspace/bgg-ranking-historicals/2...
3,281258,Sub Terra II: Inferno's Edge,2021,9152,8.58,5.553,51,/boardgame/281258/sub-terra-ii-infernos-edge,https://cf.geekdo-images.com/micro/img/dMc_drC...,file:/home/workspace/bgg-ranking-historicals/2...
4,242705,Aeon Trespass: Odyssey,2021,10006,8.53,5.543,41,/boardgame/242705/aeon-trespass-odyssey,https://cf.geekdo-images.com/micro/img/lMPwOlC...,file:/home/workspace/bgg-ranking-historicals/2...


##### Apparently here some of the years do not make sense. Having a look at the input files we can confirm that it is a mistake in the source files. Since we have the year in the meta data, we will drop this column here. We will also drop other columns which are not of interest for our purposes.

In [25]:
FILTER_COLS = ['ID',            \
               'Name',          \
               'Rank',          \
               'Average',       \
               'Bayes average', \
               'Users rated',   \
               'filename'       ]

df_staging_ranking = df_staging_ranking_unfiltered.select(FILTER_COLS)

In [26]:
df_staging_ranking.limit(5).toPandas()

,ID,Name,Rank,Average,Bayes average,Users rated,filename
0,174430,Gloomhaven,1,8.85,8.585,30740,file:/home/workspace/bgg-ranking-historicals/2...
1,161936,Pandemic Legacy: Season 1,2,8.63,8.472,34439,file:/home/workspace/bgg-ranking-historicals/2...
2,167791,Terraforming Mars,3,8.42,8.267,47598,file:/home/workspace/bgg-ranking-historicals/2...
3,182028,Through the Ages: A New Story of Civilization,4,8.50,8.236,18093,file:/home/workspace/bgg-ranking-historicals/2...
4,224517,Brass: Birmingham,5,8.62,8.197,9728,file:/home/workspace/bgg-ranking-historicals/2...


In [27]:
df_staging_ranking.count()

2243307

##### Writing to parquet only works when we have no spaces in column names

In [28]:
df_staging_ranking = df_staging_ranking.withColumnRenamed("Bayes average", "Bayes_Avg").withColumnRenamed("Users rated", "Users_rated")

In [29]:
# persisting to parquet
df_staging_ranking.write.mode("overwrite").parquet("staging_ranking_data")
df_staging_meta.write.mode("overwrite").parquet("staging_meta_data")

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
We choose a snowflake model with five tables: 

rankings, games, game_categories, game_families, game_mechanics. 

The first being the fact table, the latter the dimension tables.

##### rankings (fact table)

|   Column    |  Type    |  Property   |
| ----------- | -------- | ----------- |
| id          | integer  | Primary key |
| rank        | integer  |             |
| average     | double   |             |
| bayes_avg   | double   |             |
| users_rated | integer  |             |
| date        | datetime |             |
| game_id     | integer  |             |

##### games (dimension table)

|   Column    |  Type   |  Property   |
| ----------- | ------- | ----------- |
| game_id     | integer | Primary key |
| game_type   | varchar |             |
| maxplayers  | integer |             |
| maxplaytime | integer |             |
| minage      | integer |             |
| minplayers  | integer |             |
| minplaytime | integer |             |
| playingtime | integer |             |
| year        | integer |             |
| artist      | varchar |             |
| designer    | varchar |             |

##### game_categories (dimension table)

|   Column   |  Type   |  Property   |
| ---------- | ------- | ----------- |
| id         | integer | Primary key |
| category   | varchar |             |
| game_id    | integer |             |


##### game_mechanics (dimension table)

|   Column   |  Type   |  Property   |
| ---------- | ------- | ----------- |
| id         | integer | Primary key |
| mechanic   | varchar |             |
| game_id    | integer |             |


##### game_family (dimension table)

|   Column   |  Type   |  Property   |
| ---------- | ------- | ----------- |
| id         | integer | Primary key |
| family     | varchar |             |
| game_id    | integer |             |


#### 3.2 Mapping Out Data Pipelines
We still need to do some transformations:

1) We have to select the right columns, give an alias and perform a join on the two given tables to find the common games (name-wise).

2) The columns attributes_boardgamecategory, attributes_boardgamefamily, attributes_boardgamemechanic contain lists. We need to extract/explode that information.

3) In the ranking table, the column 'filename' tells when the ranking was collected. We need to extract this date information from the given filepath.

4) We are only interested in ratings where more meta data exists. So we can drop ratings which can not be mapped to a game in the meta data.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

##### Preparing rankings table

In [30]:
df_rankings = df_staging_ranking.join(df_staging_meta, df_staging_ranking["Name"] == df_staging_meta["details_name"], 'inner') \
                                .select(monotonically_increasing_id().alias("id"),           \
                                        df_staging_ranking.Rank.alias("rank"),               \
                                        df_staging_ranking.Average.alias("average"),         \
                                        df_staging_ranking.Bayes_Avg.alias("bayes_avg"),     \
                                        df_staging_ranking.Users_rated.alias("users_rated"), \
                                        to_date(substring_index(substring_index(df_staging_ranking.filename, "/", -1), '.', 1), 'yyyy-mm-dd').alias("date"), \
                                        df_staging_meta.game_id.alias("game_id"))

df_rankings.limit(5).toPandas()

,id,rank,average,bayes_avg,users_rated,date,game_id
0,0,6141,6.68,5.617,154,2020-01-03,147474
1,1,6136,6.68,5.617,154,2020-01-02,147474
2,2,6125,6.68,5.618,154,2020-01-01,147474
3,3,6122,6.68,5.618,154,2019-01-31,147474
4,4,6121,6.68,5.618,154,2019-01-30,147474


In [31]:
df_rankings.count()

2253038

In [32]:
df_rankings.write.partitionBy(["date"]).parquet('rankings', mode='overwrite')

##### Preparing game_details table

In [33]:
df_game_details = df_staging_meta.select(df_staging_meta.game_id.alias("game_id"), \
                                         df_staging_meta.details_maxplayers.alias("maxplayers"), \
                                         df_staging_meta.details_maxplaytime.alias("maxplaytime"), \
                                         df_staging_meta.details_minage.alias("minage"), \
                                         df_staging_meta.details_minplayers.alias("minplayers"), \
                                         df_staging_meta.details_minplaytime.alias("minplaytime"), \
                                         df_staging_meta.details_playingtime.alias("playingtime"), \
                                         df_staging_meta.details_yearpublished.alias("year"), \
                                         df_staging_meta.attributes_boardgameartist.alias("artist"), \
                                         df_staging_meta.attributes_boardgamedesigner.alias("designer"))

df_game_details.limit(5).toPandas()

,game_id,maxplayers,maxplaytime,minage,minplayers,minplaytime,playingtime,year,artist,designer
0,1,5,240,14,3,240,240,1986,Marcus Gschwendtner,Karl-Heinz Schmiel
1,2,4,30,12,3,30,30,1981,Bob Pepper,"G. W. ""Jerry"" D'Arcey"
2,3,4,60,10,2,30,60,1998,Franz Vohwinkel,Reiner Knizia
3,4,4,60,12,2,60,60,1992,None,Christian Beierer
4,5,6,90,12,3,90,90,1964,"Scott Okumura,Peter Whitley",Sid Sackson


In [34]:
df_game_details.write.parquet('game_details', mode='overwrite')

In [35]:
df_game_details.count()

90400

##### Preparing game_categories table

In [36]:
df_game_categories = df_staging_meta.select(monotonically_increasing_id().alias("id"), \
                                            explode(split(df_staging_meta.attributes_boardgamecategory, ',')).alias("category"), \
                                            df_staging_meta.game_id.alias("game_id"))

df_game_categories.limit(5).toPandas()

,id,category,game_id
0,0,Economic,1
1,1,Negotiation,1
2,2,Political,1
3,3,Card Game,2
4,4,Fantasy,2


In [37]:
df_game_categories.write.parquet('game_categories', mode='overwrite')

##### Preparing game_mechanics table

In [38]:
df_game_mechanics = df_staging_meta.select(monotonically_increasing_id().alias("id"), \
                                            explode(split(df_staging_meta.attributes_boardgamemechanic, ',')).alias("mechanic"), \
                                            df_staging_meta.game_id.alias("game_id"))

df_game_mechanics.limit(5).toPandas()

,id,mechanic,game_id
0,0,Area Control / Area Influence,1
1,1,Auction/Bidding,1
2,2,Dice Rolling,1
3,3,Hand Management,1
4,4,Simultaneous Action Selection,1


In [39]:
df_game_mechanics.write.parquet('game_mechanics', mode='overwrite')

##### Preparing game_family

In [40]:
df_game_family = df_staging_meta.select(monotonically_increasing_id().alias("id"), \
                                            explode(split(df_staging_meta.attributes_boardgamefamily, ',')).alias("family"), \
                                            df_staging_meta.game_id.alias("game_id"))

df_game_family.limit(5).toPandas()

,id,family,game_id
0,0,Country: Germany,1
1,1,Valley Games Classic Line,1
2,2,Animals: Dragons,2
3,3,Asian Theme,3
4,4,Country: Japan,3


In [41]:
df_game_family.write.parquet('game_family', mode='overwrite')

#### 4.2 Data Quality Checks

In [42]:
hasRows = (df_rankings.count() > 0)
hasRows = (df_game_details.count() > 0)    and hasRows
hasRows = (df_game_categories.count() > 0) and hasRows
hasRows = (df_game_mechanics.count() > 0)  and hasRows
hasRows = (df_game_family.count() > 0)     and hasRows

if hasRows:
    print('Success: All tables have rows!')
else:
    raise ValueError('Failure: At least one of the tables (fact or dimension tables) is empty.')

Success: All tables have rows!


In [43]:
hasUniqueKeys = (df_rankings.select("id").distinct().count() == df_rankings.count())
hasUniqueKeys = (df_game_details.select("game_id").distinct().count() == df_game_details.count())  and hasUniqueKeys
hasUniqueKeys = (df_game_categories.select("id").distinct().count() == df_game_categories.count()) and hasUniqueKeys
hasUniqueKeys = (df_game_mechanics.select("id").distinct().count() == df_game_mechanics.count())   and hasUniqueKeys
hasUniqueKeys = (df_game_family.select("id").distinct().count() == df_game_family.count())         and hasUniqueKeys

if hasUniqueKeys:
    print('Success: All tables have unique IDs!')
else:
    raise ValueError('Failure: At least one of the tables does not have unique IDs.')

Success: All tables have unique IDs!


#### 4.3 Data Dictionary

##### rankings (fact table)

|   Column    |                    description                     |
| ----------- | -------------------------------------------------- |
| id          | auto incrementing key                              | 
| rank        | rank of the game on the particular date            |
| average     | average rating                                     |
| bayes_avg   | bayes average rating                               |
| users_rated | number of users who rated                          |
| date        | time when the other values were calculated / taken |
| game_id     | foreign key: linking to game_details               |

##### game_details (dimension table)

|   Column    |                           description                         |
| ----------- | ------------------------------------------------------------- |
| game_id     | auto incrementing key                                         |
| game_type   | which type of game; e.g. board game, board game expansion, .. |
| maxplayers  | maximum number of players                                     |
| maxplaytime | maximum playtime for one game                                 |
| minage      | recommended minimum age of players                            |
| minplayers  | minimum number of players                                     |
| minplaytime | minimum playtime for one game                                 |
| playingtime | average playing time for one game                             |
| year        | year in which the game was published                          |
| artist      | game artist                                                   |
| designer    | game designer                                                 |

##### game_categories (dimension table)

|   Column   |                 description                 |
| ---------- | ------------------------------------------- |
| id         | auto incrementing key                       |
| category   | game category (e.g. Political, Fantasy, ..) |
| game_id    | foreign key: linking to game_details        |


##### game_mechanics (dimension table)

|   Column   |                      description                        |
| ---------- | ------------------------------------------------------ |
| id         | auto incrementing key                                  |
| mechanic   | game mechanic (e.g. Auction/Bidding, Dice Rolling, ..) |
| game_id    | foreign key: linking to game_details                   |


##### game_family (dimension table)

|   Column   |               description                  |
| ---------- | ------------------------------------------ |
| id         | auto incrementing key                      |
| family     | game family (e.g. Asian Theme, Dragon, ..) |
| game_id    | foreign key: linking to game_details       |

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.

We decided to use Spark since in case data was significally increased, we would already be able to use the parallelism of spark without much rewriting.
We further decided to stay on a local machine as long as the amount of data is managable. The data model is locally persisted in parquet files (the large dataset rankings is partioned by date). This could easily be transferred to S3 in future in case we have to deal with more data.

* Propose how often the data should be updated and why.

The data source for getting the rankings is updated once a day. So it would make sense to receive these updates on a daily basis to be able to recognize trends as early as possible.

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In case the data would be increased by 100x, we most likely will lack performance on our local machine. Since the code is already prepared and written for the usage of Spark, I would then suggest to run this notebook on a cluster (e.g. AWS) and populate the data to S3.

In case the data populates a dashboard once a day, I would suggest to write an airflow pipeline which first pulls the rankings from git and then reloads the meta data from kaggle via API. After that the pipeline should do the necessary transformations and persist the data. This enables us to recognize errors faster and since this would be an automated process I would additionally suggest to write more tests to ensure data quality.

In case the database needs to be accessed by 100+ people, I would also suggest to move the storage to AWS (or a similar service).